In [1]:
#To allocate memorey for gpu(here we are using 50%)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [8]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('new_data.json').read()
intents = json.loads(data_file)

In [9]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['We offer lot courses on

In [10]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [11]:
print(words)

['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'What', 'are', 'the', 'courses', 'do', 'you', 'provide', '?', 'Can', 'you', 'tell', 'me', 'the', 'best', 'language', 'ever', '?', 'What', 'language', 'can', 'I', 'start', 'for', 'programming', 'I', 'am', 'a', 'beginner', 'and', 'I', 'need', 'to', 'learn', 'a', 'new', 'language', 'What', 'are', 'the', 'courses', 'do', 'you', 'provide', 'on', 'python', '?', 'Waht', 'is', 'duration', 'of', 'Course', 'for', 'Core', 'and', 'Advance', 'Python', '?', 'What', 'is', 'the', 'Cost', 'fo

In [12]:
print(classes)

['greeting', 'goodbye', 'thanks', 'options', 'courses_offered', 'Python', 'PythonDuration', 'PythonCost', 'PythonMentor', 'PythonSyllabus', 'DataScience', 'DataScienceDuration', 'DataScienceFees', 'DataScienceMentor', 'DataScienceSyllabus', 'Java', 'JavaDuration', 'JavaFees', 'JavaMentor', 'JavaSyllabus', 'JavaFrameworks', 'JavaFrameworksDuration', 'JavaFrameworksFees', 'JavaFrameworksMentor', 'WebDevelopmentPython', 'WebDevelopmentPythonDuration', 'WebDevelopmentPythonFees', 'WebDevelopmentPythonMentor', 'WebDevelopmentJava', 'WebDevelopmentJavaDuration', 'WebDevelopmentJavaFees', 'WebDevelopmentJavaMentor']


In [13]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [14]:
print(words)

["'s", ',', 'a', 'advance', 'advanced', 'am', 'and', 'anyone', 'are', 'awesome', 'be', 'beginner', 'best', 'bye', 'can', 'chatting', 'core', 'corse', 'cost', 'could', 'course', 'data', 'day', 'development', 'do', 'duration', 'ever', 'for', 'framework', 'full', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'java', 'javaframeworks', 'language', 'later', 'learn', 'me', 'mentor', 'need', 'new', 'next', 'nice', 'of', 'offered', 'on', 'programming', 'provide', 'python', 'science', 'see', 'start', 'support', 'syllabus', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'waht', 'web', 'what', 'who', 'with', 'you']


In [15]:
# sort classes
classes = sorted(list(set(classes)))

In [16]:
print(classes)

['DataScience', 'DataScienceDuration', 'DataScienceFees', 'DataScienceMentor', 'DataScienceSyllabus', 'Java', 'JavaDuration', 'JavaFees', 'JavaFrameworks', 'JavaFrameworksDuration', 'JavaFrameworksFees', 'JavaFrameworksMentor', 'JavaMentor', 'JavaSyllabus', 'Python', 'PythonCost', 'PythonDuration', 'PythonMentor', 'PythonSyllabus', 'WebDevelopmentJava', 'WebDevelopmentJavaDuration', 'WebDevelopmentJavaFees', 'WebDevelopmentJavaMentor', 'WebDevelopmentPython', 'WebDevelopmentPythonDuration', 'WebDevelopmentPythonFees', 'WebDevelopmentPythonMentor', 'courses_offered', 'goodbye', 'greeting', 'options', 'thanks']


In [17]:
print(classes)

['DataScience', 'DataScienceDuration', 'DataScienceFees', 'DataScienceMentor', 'DataScienceSyllabus', 'Java', 'JavaDuration', 'JavaFees', 'JavaFrameworks', 'JavaFrameworksDuration', 'JavaFrameworksFees', 'JavaFrameworksMentor', 'JavaMentor', 'JavaSyllabus', 'Python', 'PythonCost', 'PythonDuration', 'PythonMentor', 'PythonSyllabus', 'WebDevelopmentJava', 'WebDevelopmentJavaDuration', 'WebDevelopmentJavaFees', 'WebDevelopmentJavaMentor', 'WebDevelopmentPython', 'WebDevelopmentPythonDuration', 'WebDevelopmentPythonFees', 'WebDevelopmentPythonMentor', 'courses_offered', 'goodbye', 'greeting', 'options', 'thanks']


In [18]:
pickle.dump(words,open('words_old.pkl','wb'))
pickle.dump(classes,open('classes_old.pkl','wb'))

In [19]:
# create our training data
training = []

In [20]:
# create an empty array for our output
output_empty = [0] * len(classes)

In [21]:
# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [22]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

C:\ProgramData\Anaconda3\envs\gpu\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0]) # patterns
train_y = list(training[:,1]) #responses

In [24]:
# Create model
model = Sequential()
#First layer 128 neurons
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
#Second layer 128 neurons
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
#output layer
model.add(Dense(len(train_y[0]), activation='softmax'))

In [25]:
#compile model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [26]:
#fitting the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=250, batch_size=5, verbose=1)

Epoch 1/250
53/53 [==============================] - 6s 113ms/step - loss: 3.5084 - accuracy: 0.0189
Epoch 2/250
53/53 [==============================] - 0s 1ms/step - loss: 3.4359 - accuracy: 0.0566
Epoch 3/250
53/53 [==============================] - 0s 1ms/step - loss: 3.3373 - accuracy: 0.1132
Epoch 4/250
53/53 [==============================] - 0s 2ms/step - loss: 3.2464 - accuracy: 0.1887
Epoch 5/250
53/53 [==============================] - 0s 2ms/step - loss: 3.1822 - accuracy: 0.1698
Epoch 6/250
53/53 [==============================] - 0s 2ms/step - loss: 3.0464 - accuracy: 0.2453
Epoch 7/250
53/53 [==============================] - 0s 1ms/step - loss: 2.9192 - accuracy: 0.3208
Epoch 8/250
53/53 [==============================] - 0s 1ms/step - loss: 2.8816 - accuracy: 0.2642
Epoch 9/250
53/53 [==============================] - 0s 1ms/step - loss: 2.7001 - accuracy: 0.2830
Epoch 10/250
53/53 [==============================] - 0s 1ms/step - loss: 2.5516 - accuracy: 0.3396
Epoch 1

53/53 [==============================] - 0s 679us/step - loss: 0.5669 - accuracy: 0.7736
Epoch 83/250
53/53 [==============================] - 0s 698us/step - loss: 0.5890 - accuracy: 0.7736
Epoch 84/250
53/53 [==============================] - 0s 698us/step - loss: 0.5072 - accuracy: 0.7736
Epoch 85/250
53/53 [==============================] - 0s 679us/step - loss: 0.5033 - accuracy: 0.7736
Epoch 86/250
53/53 [==============================] - 0s 661us/step - loss: 0.5479 - accuracy: 0.8679
Epoch 87/250
53/53 [==============================] - 0s 679us/step - loss: 0.4430 - accuracy: 0.8302
Epoch 88/250
53/53 [==============================] - 0s 698us/step - loss: 0.5576 - accuracy: 0.8113
Epoch 89/250
53/53 [==============================] - 0s 679us/step - loss: 0.3773 - accuracy: 0.8302
Epoch 90/250
53/53 [==============================] - 0s 679us/step - loss: 0.6332 - accuracy: 0.7736
Epoch 91/250
53/53 [==============================] - 0s 661us/step - loss: 0.5367 - accuracy: 

53/53 [==============================] - 0s 717us/step - loss: 0.4857 - accuracy: 0.7925
Epoch 162/250
53/53 [==============================] - 0s 698us/step - loss: 0.3795 - accuracy: 0.8113
Epoch 163/250
53/53 [==============================] - 0s 698us/step - loss: 0.3107 - accuracy: 0.9057
Epoch 164/250
53/53 [==============================] - 0s 698us/step - loss: 0.2496 - accuracy: 0.8868
Epoch 165/250
53/53 [==============================] - 0s 698us/step - loss: 0.3006 - accuracy: 0.9245
Epoch 166/250
53/53 [==============================] - 0s 698us/step - loss: 0.3193 - accuracy: 0.9245
Epoch 167/250
53/53 [==============================] - 0s 698us/step - loss: 0.2040 - accuracy: 0.9245
Epoch 168/250
53/53 [==============================] - 0s 698us/step - loss: 0.3702 - accuracy: 0.8302
Epoch 169/250
53/53 [==============================] - 0s 698us/step - loss: 0.3006 - accuracy: 0.9245
Epoch 170/250
53/53 [==============================] - 0s 698us/step - loss: 0.3330 - a

53/53 [==============================] - 0s 717us/step - loss: 0.1521 - accuracy: 0.9245
Epoch 241/250
53/53 [==============================] - 0s 698us/step - loss: 0.3522 - accuracy: 0.8491
Epoch 242/250
53/53 [==============================] - 0s 717us/step - loss: 0.2659 - accuracy: 0.8868
Epoch 243/250
53/53 [==============================] - 0s 736us/step - loss: 0.3576 - accuracy: 0.8868
Epoch 244/250
53/53 [==============================] - 0s 679us/step - loss: 0.2402 - accuracy: 0.9434
Epoch 245/250
53/53 [==============================] - 0s 679us/step - loss: 0.2441 - accuracy: 0.9057
Epoch 246/250
53/53 [==============================] - 0s 679us/step - loss: 0.1311 - accuracy: 0.9245
Epoch 247/250
53/53 [==============================] - 0s 698us/step - loss: 0.2443 - accuracy: 0.9057
Epoch 248/250
53/53 [==============================] - 0s 680us/step - loss: 0.2996 - accuracy: 0.8868
Epoch 249/250
53/53 [==============================] - 0s 680us/step - loss: 0.3094 - a

In [27]:
# save model
model.save('old_chatbot_model.h5', hist)